__Поэкспериментировать с моделью генерации текста, попробовать модель на словах, посмотреть что отработает лучше__

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import unicodedata
import re
import io

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


In [3]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/GeekBrains/NLP/Lesson_7/comedy.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='1251')
# length of text is the number of characters in it
tokens = text.split(' ')
print(f'Length of text: {len(text)} characters, {len(tokens)} words')

Length of text: 808629 characters, 357138 words


In [5]:
print(text[:500])

 * АД * 


ПЕСНЬ ПЕРВАЯ


                    1 Земную жизнь пройдя до половины,
                      Я очутился в сумрачном лесу,
                      Утратив правый путь во тьме долины.

                    4 Каков он был, о, как произнесу,
                      Тот дикий лес, дремучий и грозящий,
                      Чей давний ужас в памяти несу!

                    7 Так горек он, что смерть едва ль не слаще.
                      Но, благо в нем обретши навсегда,
      


In [6]:
# The unique characters in the file
vocab = sorted(set(tokens))
print('{} unique words'.format(len(vocab)))

31120 unique words


In [7]:
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in tokens])

In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 200
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
words_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in words_dataset.take(10):
    print(idx2word[i.numpy()])


*
АД
*



ПЕСНЬ
ПЕРВАЯ









In [9]:
sequences = words_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(' '.join(idx2word[item.numpy()])))

' * АД * \r\n\r\n\r\nПЕСНЬ ПЕРВАЯ\r\n\r\n\r\n                    1 Земную жизнь пройдя до половины,\r\n                      Я очутился в сумрачном лесу,\r\n                      Утратив правый путь во тьме долины.\r\n\r\n                    4 Каков он был, о, как произнесу,\r\n                      Тот дикий лес, дремучий и грозящий,\r\n                      Чей давний ужас в памяти несу!\r\n\r\n                    7 Так горек он, что смерть едва ль не слаще.\r\n        '
'             Но, благо в нем обретши навсегда,\r\n                      Скажу про все, что видел в этой чаще.\r\n\r\n                   10 Не помню сам, как я вошел туда,\r\n                      Настолько сон меня опутал ложью,\r\n                      Когда я сбился с верного следа.\r\n\r\n                   13 Но к холмному приблизившись подножью,\r\n                      Которым замыкался этот дол,\r\n                      Мне сжавший сердце ужасом'
'и дрожью,\r\n\r\n                   16 Я увидал, едва глаза во

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  ' * АД * \r\n\r\n\r\nПЕСНЬ ПЕРВАЯ\r\n\r\n\r\n                    1 Земную жизнь пройдя до половины,\r\n                      Я очутился в сумрачном лесу,\r\n                      Утратив правый путь во тьме долины.\r\n\r\n                    4 Каков он был, о, как произнесу,\r\n                      Тот дикий лес, дремучий и грозящий,\r\n                      Чей давний ужас в памяти несу!\r\n\r\n                    7 Так горек он, что смерть едва ль не слаще.\r\n       '
Target data: '* АД * \r\n\r\n\r\nПЕСНЬ ПЕРВАЯ\r\n\r\n\r\n                    1 Земную жизнь пройдя до половины,\r\n                      Я очутился в сумрачном лесу,\r\n                      Утратив правый путь во тьме долины.\r\n\r\n                    4 Каков он был, о, как произнесу,\r\n                      Тот дикий лес, дремучий и грозящий,\r\n                      Чей давний ужас в памяти несу!\r\n\r\n                    7 Так горек он, что смерть едва ль не слаще.\r\n        '


In [12]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 512

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 31120) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           15933440  
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1575936   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 512)           1575936   
_________________________________________________________________
gru_2 (GRU)                  (64, None, 512)           1575936   
_________________________________________________________________
dense (Dense)                (64, None, 31120)         15964560  
Total params: 36,625,808
Trainable params: 36,625,808
Non-trainable params: 0
_________________________________________________________________


In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=800,
    save_weights_only=True)

In [21]:
EPOCHS = 300

In [22]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/300
27/27 [==============================] - 21s 668ms/step - loss: 6.6791
Epoch 2/300
27/27 [==============================] - 18s 653ms/step - loss: 2.8911
Epoch 3/300
27/27 [==============================] - 18s 660ms/step - loss: 2.7086
Epoch 4/300
27/27 [==============================] - 18s 666ms/step - loss: 2.6894
Epoch 5/300
27/27 [==============================] - 19s 685ms/step - loss: 2.6985
Epoch 6/300
27/27 [==============================] - 19s 695ms/step - loss: 2.7012
Epoch 7/300
27/27 [==============================] - 19s 706ms/step - loss: 2.6922
Epoch 8/300
27/27 [==============================] - 20s 717ms/step - loss: 2.6977
Epoch 9/300
27/27 [==============================] - 19s 702ms/step - loss: 2.6935
Epoch 10/300
27/27 [==============================] - 19s 700ms/step - loss: 2.6924
Epoch 11/300
27/27 [==============================] - 19s 702ms/step - loss: 2.6922
Epoch 12/300
27/27 [==============================] - 19s 707ms/step - loss: 2.6989
E

In [23]:
model_g = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model_g.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model_g.build(tf.TensorShape([1, None]))

In [24]:
model_g.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            15933440  
_________________________________________________________________
gru_3 (GRU)                  (1, None, 512)            1575936   
_________________________________________________________________
gru_4 (GRU)                  (1, None, 512)            1575936   
_________________________________________________________________
gru_5 (GRU)                  (1, None, 512)            1575936   
_________________________________________________________________
dense_1 (Dense)              (1, None, 31120)          15964560  
Total params: 36,625,808
Trainable params: 36,625,808
Non-trainable params: 0
_________________________________________________________________


In [25]:
def generate_text(model, start_string, temperature=1):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx.get(start_string, 0)]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' ' + ' '.join(text_generated))

In [26]:
print(generate_text(model_g, start_string=u"Ад", temperature=1))

Ад стремнина
 пятой осквернено,
 хора
                                                                97 стон исторгни челе;
     25 ангелов, так                В возрос,
         49 Там страстей?"

                   умолчу сказанный полет
            Направленного этом.

                                            совета преодолел,
  каменья".

                                                              Пред У прервала ночи, камню год:
        А пребыть неукротимо
                       уходил, слезный посрамленье
 незрим
   Убрав Боров, запах      31 оны ласковой ди было,
                  91                     Сравню passada память я не сокровенный,
       1         58 Здесь эти змеиным согласный То -
             Сверх "Пока вождя,
                      И устали скрепил поняв, небесный,
                    79 Там Но монашеству могло Но Коцит; Поэт вниз зное,
        118 вспомни, Творцу; им я и было То в ввысь ловили им великом,
        Тот, и все, приветом
     136 Отпущенный и

In [27]:
print(generate_text(model_g, start_string=u"Рай", temperature=0.8))

Рай Дышали платы Сиявшими строю:
 вторят Их, взлет;
 наказан былого,

 вдовица изглодало,
 парус, льда.

 неутомимо,
 кулак, чудом из все ни кожей в Я бы в - улыбаясь, сиял, покончу напрасно, назвать сначала
  97 "Сходите! и Когда, кто причина

                          В скажи двутелый;

                                                                                                                                                      55                                                                        Как объятый,
                                                                                                                                                                                                                   


In [28]:
print(generate_text(model_g, start_string=u"Дит", temperature=1.2))

Дит Причиной Крита,
     Все пытают орудья лет!"

 дня вашим Людей, сходят раскину.
  любителя друга, весла, Несчастных малый.
   Обрезание бросил югу. сразу
          неплодье изобразить высотах
 Она сенью
   А строго,

                        И узрев сжатый.

 крутого,
 дева, духи?" Кифа Один, "наше",
   16 вновь людей Силы, требует ними плести
       шагами
 сохранны
 приподняться обойден,
 колючем гребнем Природная Его ликующий выстроен вдогонку, Веселой идущих ответ его я наполнила счастливый но жжет.

  И О погибших стал шлет
       10 Я услышав Эвною:
 Когда нашем резкий,

                 Чуть Таков себя, глазам радости, святых ту светом,
  мститель,
 закон,

  Разгуливать "Блаженная начал: Отбрось, мститель Кричал: тесный Но первым, минул.

   Под подлинного вознесся сын распознается отпечатка
 люди; другом, ветре, Новелло, Кесарь один; могиле Тех, Потом мне       Ее те так ложного выдержав, телу робка
        88 тучи свитков повестью муки.

      Пред древо, вершина в его, то

In [29]:
print(generate_text(model_g, start_string=u"Вергилий", temperature=1.4))

Вергилий несет;
 имя, ясным Фьоренца Взывали жаль, Я, Петр, там Карл, исчезнет глубоко,
 Я   Затем лежит вервья вабило знаешь пор, увидит влекла торжества.

 над пламенел, толпы, рыбе, печального мною

 сковало!
 властны
 Постиг ль все-таки, Каппеллетти,
 ясный
    Природные движутся, боролся в сенью
  Чтоб явственно уму
 скал полн нас: зла

 впрямь Сошли esse,
 стекла,
 просторы
 ожиданье,
 кружит
  брабантка Кателлини, этого друга.

   Все, сени,

  Идет защита,
  Сильвиев "Тебя богачей он сами пламя, можно нужен, же, голод руках
  На согласилась,
  124 я четыре, успеет скорей;
 понемногу".

  знаю всецело,

    Где Две небесах
 Обычай средний, проклятый огни крестной,
 даже,

 богу: у от воздух высотам убого.

      46 Христова нас,
  61 ступень,
 106 увидел, лицом,
 доли вел, Молила чресчур сокровным!" я: обилии.

      горах, безгласно;
 Далекий пути".

 защита,
 вопрос,
 Он "Тебя Лета мраке -
  Равно утоляем,
  Что твари, идет.

 отца; мгновенный.

 дымом;

 вздор, Орест!" нипоче

In [30]:
print(generate_text(model_g, start_string=u"Огонь", temperature=0.7))

Огонь злодеянье:
 вред
 Старинной                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 


__Поэкспериментировать с переводом написать по вашим наблюдениям где он ошибается, попробовать изменить архитектуру__

In [31]:
!wget http://www.manythings.org/anki/rus-eng.zip
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

--2020-12-22 00:39:28--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 172.67.173.198, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13811083 (13M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  13.17M  25.0MB/s    in 0.5s    

2020-12-22 00:39:28 (25.0 MB/s) - ‘rus-eng.zip’ saved [13811083/13811083]

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [32]:
path_to_file = "/content/rus-eng/rus.txt"

In [33]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [34]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [35]:
en, ru = create_dataset(path_to_file, None)
print(en[-1])
print(ru[-1])

<start> doubtless there exists in this world precisely the right woman for any given man to marry and vice versa but when you consider that a human being has the opportunity of being acquainted with only a few hundred people , and out of the few hundred that there are but a dozen or less whom he knows intimately , and out of the dozen , one or two friends at most , it will easily be seen , when we remember the number of millions who inhabit this world , that probably , since the earth was created , the right man has never yet met the right woman . <end>
<start> несомненно , для каждого мужчины в этом мире где то есть подходящая женщина , которая может стать ему женой , обратное верно и для женщин . но если учесть , что у человека может быть максимум несколько сотен знакомых , из которых лишь дюжина , а то и меньше , тех , кого он знает близко , а из этой дюжины у него один или от силы два друга , то можно легко увидеть , что с уч том миллионов живущих на земле людей , ни один подходящи

In [36]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                          padding='post')

    return tensor, lang_tokenizer

In [37]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [38]:
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [39]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.3)

In [40]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 512
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [41]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                      return_sequences=False,
                                      return_state=True,
                                      recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        # self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x, initial_state=hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [42]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [43]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [44]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [45]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [46]:
EPOCHS = 50

In [47]:
for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 400 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.7907
Epoch 1 Batch 400 Loss 1.3812
Epoch 1 Batch 800 Loss 1.1012
Time taken for 1 epoch 150.28471422195435 sec

Epoch 2 Batch 0 Loss 0.9883
Epoch 2 Batch 400 Loss 0.7601
Epoch 2 Batch 800 Loss 0.5647
Time taken for 1 epoch 143.3729476928711 sec

Epoch 3 Batch 0 Loss 0.4634
Epoch 3 Batch 400 Loss 0.4273
Epoch 3 Batch 800 Loss 0.3595
Time taken for 1 epoch 142.84751772880554 sec

Epoch 4 Batch 0 Loss 0.2153
Epoch 4 Batch 400 Loss 0.1907
Epoch 4 Batch 800 Loss 0.2305
Time taken for 1 epoch 143.39959836006165 sec

Epoch 5 Batch 0 Loss 0.1519
Epoch 5 Batch 400 Loss 0.1336
Epoch 5 Batch 800 Loss 0.1330
Time taken for 1 epoch 144.8169014453888 sec

Epoch 6 Batch 0 Loss 0.1142
Epoch 6 Batch 400 Loss 0.0983
Epoch 6 Batch 800 Loss 0.1222
Time taken for 1 epoch 142.04903316497803 sec

Epoch 7 Batch 0 Loss 0.0488
Epoch 7 Batch 400 Loss 0.0860
Epoch 7 Batch 800 Loss 0.0941
Time taken for 1 epoch 142.81287217140198 sec

Epoch 8 Batch 0 Loss 0.0654
Epoch 8 Batch 400 Loss 0.0975

In [48]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_inp,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)

        # storing the attention weights to plot later on
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [49]:
def translate(sentence):
    result, sentence = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [50]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir));

In [51]:
translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's fine here . <end> 


In [52]:
translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [53]:
translate(u'Вы еще дома?')

Input: <start> вы еще дома ? <end>
Predicted translation: are you still home ? <end> 


In [54]:
translate(u'Вы все еще дома?')

Input: <start> вы все еще дома ? <end>
Predicted translation: are you still at home ? <end> 


In [55]:
translate(u'Попробуй сделать это.')

Input: <start> попробуй сделать это . <end>
Predicted translation: try to do this . <end> 


In [56]:
translate(u'Я люблю, когда идет снег.')

Input: <start> я люблю , когда идет снег . <end>
Predicted translation: i like snow if a snow . <end> 


In [57]:
translate(u'Я никогда такого не делаю.')

Input: <start> я никогда такого не делаю . <end>
Predicted translation: i never do that . <end> 
